In [36]:
# Import dependencies
import os
import subprocess
import json
from dotenv import load_dotenv
from pprint import pprint
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI

In [37]:
# Load and set environment variables
load_dotenv("mnemonic.env")
mnemonic=os.getenv("mnemonic")

In [38]:
#Import constants.py and necessary functions from bit and web3
from constants import *

In [39]:
# Create a function called `derive_wallets`
def derive_wallets(coin=BTC, mnemonic=mnemonic, depth=3):
    command = f'php ./derive -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive={depth} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)
    
 

In [40]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {
    ETH: derive_wallets(coin=ETH),
    BTCTEST: derive_wallets(coin=BTCTEST),
}

pprint(coins)

{'btc-test': [{'address': 'mm3bDoBni6znWYRVjsJfpiN6LmKgdtgmH8',
               'index': 0,
               'path': "m/44'/1'/0'/0/0",
               'privkey': 'cVWYKPgryPS26jfqq49fYRt2sTKoYMz9PqeJPjVk1irJuTNR1krQ',
               'pubkey': '039ebc2ea3ad71a562731db8a83144de7a591e4ad683263218900af15c5586dcec',
               'pubkeyhash': '3ca3eebb9e3e1a3e249f40138c189558e4931eef',
               'xprv': 'tprv8jxnove2ZJDyt2UkbsW8fo2K2VM4C1BC8k9oyAYSKwf9sVZQMgmd2H1CQJZTQS9RqhiixkSLH46KqNVn5QbAzoXbXui172mJ5ScMZS3Uirk',
               'xpub': 'tpubDGepxLgGhfuemVWYVXAj5CgRbWrzMLN6i3kbFgajkDTYhypAz5bDCmd4aSrpeJjmUVY6sMJTu59Q4Bd3CCEKGqqi9p6H7XLVgNa16zpEBJL'},
              {'address': 'mmXiEwfQuLiPWKj5fRvnW38Eu3PHAz3st9',
               'index': 1,
               'path': "m/44'/1'/0'/0/1",
               'privkey': 'cW8KFFKuULFZzWipHNaGYfkJ449hBpoAmjP7nRkMrWsjL9iV9A62',
               'pubkey': '02d3a007c4b700f7f47e23a572c2278d4ae6426ee0fbbc913a925b2493b6a96eff',
               'pubkeyhash': '

In [41]:

# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [42]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, receipient, amount):
    if coin == ETH:
        value = w3.toWei(amount, "ether") 
        gasEstimate = w3.eth.estimateGas({ "to": receipient, "from": account, "amount": value })
        return {
            "to": receipient,
            "from": account,
            "value": value,
            "gas": gasEstimate,
            "gasPrice": w3.eth.generateGasPrice(),
            "nonce": w3.eth.getTransactionCount(account),
            "chainId": w3.eth.chain_id
        }
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(receipient, amount, BTC)])

In [43]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, receipient, amount):
    if coin == ETH:
        raw_tx = create_tx(coin, account.address, receipient, amount)
        signed = account.signTransaction(raw_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    if coin == BTCTEST:
        raw_tx = create_tx(coin, account, receipient, amount)
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)

In [45]:
#Testing BTCTEST 
coin=BTCTEST
account = priv_key_to_account(BTCTEST, coins[BTCTEST][0]['privkey'])
receipient= 'mmXiEwfQuLiPWKj5fRvnW38Eu3PHAz3st9'
amount=0.00001
send_tx(coin, account, receipient, amount)
